In [4]:
cd ..

/Users/wesgurnee/Documents/mechint/sparse_probing


In [5]:
import os
os.environ['TRANSFORMERS_CACHE'] = '/Users/wesgurnee/Documents/mechint/sparse_probing/sparse-probing/downloads'
os.environ['TRANSFORMERS_OFFLINE'] = '1'

In [6]:
import datasets
from datasets import load_dataset
import matplotlib.pyplot as plt

%matplotlib inline

In [7]:
test_dataset = datasets.load_dataset(
    "the_pile", split="test", streaming=True)

No config specified, defaulting to: the_pile/all


In [8]:
subset = test_dataset.filter(lambda example: example['meta']['pile_set_name'] == 'ArXiv')
arxiv_test = datasets.Dataset.from_list(list(subset.take(5000)))

In [12]:
valid_dataset = datasets.load_dataset(
    "the_pile", split="validation", streaming=True)

subset = valid_dataset.filter(lambda example: example['meta']['pile_set_name'] == 'ArXiv')
arxiv_valid = datasets.Dataset.from_list(list(subset.take(5000)))

No config specified, defaulting to: the_pile/all


In [17]:
# concat the two datasets
arxiv_dataset = datasets.concatenate_datasets([arxiv_test, arxiv_valid])

In [19]:
arxiv_dataset.save_to_disk(os.path.join(os.environ.get('TRANSFORMERS_CACHE'), 'arxiv_dataset.hf'))

Saving the dataset (0/1 shards):   0%|          | 0/4841 [00:00<?, ? examples/s]

In [ ]:
train_dataset = datasets.load_dataset(
    "the_pile", split="train", streaming=True)

subset = train_dataset.filter(lambda example: example['meta']['pile_set_name'] == 'ArXiv')
arxiv_train = datasets.Dataset.from_list(list(subset.take(5000)))

In [22]:
import numpy as np
igk = 5
ctx_len = 16
valid_negatives = np.ones((100, ctx_len), dtype=bool)
valid_negatives & (
        np.arange(ctx_len) > igk)[None, :]

array([[False, False, False, ...,  True,  True,  True],
       [False, False, False, ...,  True,  True,  True],
       [False, False, False, ...,  True,  True,  True],
       ...,
       [False, False, False, ...,  True,  True,  True],
       [False, False, False, ...,  True,  True,  True],
       [False, False, False, ...,  True,  True,  True]])

In [3]:
test_dataset = datasets.load_dataset(
    "the_pile", split="test", streaming=True)

No config specified, defaulting to: the_pile/all


In [14]:
subset = datasets.Dataset.from_list(list(test_dataset.take(500_000)))

In [15]:
from collections import Counter
Counter([c['pile_set_name'] for c in subset['meta']])

Counter({'Wikipedia (en)': 17511,
         'Pile-CC': 52790,
         'PubMed Abstracts': 29895,
         'StackExchange': 30378,
         'Github': 18195,
         'OpenWebText2': 32925,
         'ArXiv': 2407,
         'USPTO Backgrounds': 11415,
         'YoutubeSubtitles': 342,
         'FreeLaw': 5101,
         'PubMed Central': 5911,
         'DM Mathematics': 1922,
         'HackerNews': 1632,
         'NIH ExPorter': 1884,
         'OpenSubtitles': 642,
         'Enron Emails': 1010,
         'EuroParl': 157,
         'Books3': 269,
         'PhilPapers': 68,
         'Gutenberg (PG-19)': 80,
         'BookCorpus2': 28,
         'Ubuntu IRC': 22})

In [18]:
save_path = '/Users/wesgurnee/Documents/mechint/sparse_probing/sparse-probing/downloads/pile-test.hf'
subset.save_to_disk(save_path)

Saving the dataset (0/3 shards):   0%|          | 0/214584 [00:00<?, ? examples/s]

In [17]:
len(subset)

214584

In [12]:
l = 0
for row in subset:
    l += len(row['text'])

In [13]:
l

614947263

In [42]:
subset = test_dataset.filter(lambda example: example['meta']['pile_set_name'] == 'Enron Emails')
enrov_valid_dataset = datasets.Dataset.from_list(list(subset.take(1000)))

In [43]:
enrov_valid_dataset

Dataset({
    features: ['text', 'meta'],
    num_rows: 947
})

In [24]:
from collections import Counter
c = (Counter([m['pile_set_name'] for m in subset['meta']]))

In [26]:
c

Counter({'Wikipedia (en)': 782,
         'Pile-CC': 2490,
         'PubMed Abstracts': 1370,
         'StackExchange': 1475,
         'Github': 825,
         'OpenWebText2': 1510,
         'ArXiv': 110,
         'USPTO Backgrounds': 506,
         'YoutubeSubtitles': 16,
         'FreeLaw': 243,
         'PubMed Central': 304,
         'DM Mathematics': 83,
         'HackerNews': 92,
         'NIH ExPorter': 78,
         'OpenSubtitles': 36,
         'Enron Emails': 45,
         'EuroParl': 8,
         'Books3': 14,
         'PhilPapers': 5,
         'Gutenberg (PG-19)': 4,
         'BookCorpus2': 2,
         'Ubuntu IRC': 2})

In [27]:
dataset_splits = {
    'Wikipedia (en)': 'wikipedia',
    'PubMed Abstracts': 'pubmed_abstracts',
    'StackExchange': 'stack_exchange',
    'Github': 'github',
    'ArXiv': 'arxiv',
    'USPTO Backgrounds': 'uspto',
    'FreeLaw': 'freelaw',
    'HackerNews': 'hackernews',
    'Enron Emails': 'enron'
}

In [34]:
test_splits = {}
split_size = 2000
for split in dataset_splits:
    subset = test_dataset.filter(lambda example: example['meta']['pile_set_name'] == split)
    sub_dataset = datasets.Dataset.from_list(list(subset.take(split_size)))
    print(split, len(sub_dataset))
    test_splits[split] = sub_dataset

Wikipedia (en) 2000
PubMed Abstracts 2000
StackExchange 2000
Github 2000
ArXiv 2000
USPTO Backgrounds 2000
FreeLaw 2000
HackerNews 1632
Enron Emails 1010


In [36]:
sit_awareness_dataset = datasets.concatenate_datasets(list(test_splits.values()))


In [46]:
sit_awareness_dataset

Dataset({
    features: ['text', 'meta'],
    num_rows: 16642
})

In [47]:
final_sit_awareness_ds = datasets.concatenate_datasets([sit_awareness_dataset, enrov_valid_dataset])

In [48]:
final_sit_awareness_ds

Dataset({
    features: ['text', 'meta'],
    num_rows: 17589
})

In [49]:
save_path = '/Users/wesgurnee/Documents/mechint/sparse_probing/sparse-probing/downloads/test_subset_with_even_slices.hf'
final_sit_awareness_ds.save_to_disk(save_path)

Saving the dataset (0/1 shards):   0%|          | 0/17589 [00:00<?, ? examples/s]

In [50]:
test_splits.keys()

dict_keys(['Wikipedia (en)', 'PubMed Abstracts', 'StackExchange', 'Github', 'ArXiv', 'USPTO Backgrounds', 'FreeLaw', 'HackerNews', 'Enron Emails'])

In [89]:
print(test_splits['Enron Emails'][1000]['text'])

Wednesday will work best for me.  Say 11:45 at the Daily Review?

-----Original Message-----
From: Jewell, Cyndi M [mailto:cjewell@bcm.tmc.edu]
Sent: Monday, February 25, 2002 8:45 AM
To: Shelley A. Corman (E-mail)
Subject: Lunch plans


Hi! I hope you enjoyed the rest of your weekend. The weather was simply
beautiful. We did some extensive yardwork. Luckily we didn't plant much
since it is supposed to be chilly this week.

Brianna and I loved the sing-a-long Sound of Music. It was great fun. I'd go
again next weekend!

Let me know which day is best for us to have lunch --Wednesday or Thursday.

C


In [4]:
save_path = '/Users/wesgurnee/Documents/mechint/sparse_probing/sparse-probing/downloads/pile-test.hf'
ds = datasets.load_from_disk(save_path)

/Users/wesgurnee/Documents/mechint/sparse_probing/sparse-probing/sparprob/lib/python3.9/site-packages/datasets/arrow_dataset.py:1533: FutureWarning: 'fs' was is deprecated in favor of 'storage_options' in version 2.8.0 and will be removed in 3.0.0.
You can remove this warning by passing 'storage_options=fs.storage_options' instead.
  warnings.warn(


In [5]:
import numpy as np
data_split = np.array([m['pile_set_name'] for m in ds['meta']])

In [17]:
np.unique(data_split)[0]

'ArXiv'

In [6]:
for k, v in zip(*np.unique(data_split, return_counts=True)):
    print(k, v)

ArXiv 2407
BookCorpus2 28
Books3 269
DM Mathematics 1922
Enron Emails 1010
EuroParl 157
FreeLaw 5101
Github 18195
Gutenberg (PG-19) 80
HackerNews 1632
NIH ExPorter 1884
OpenSubtitles 642
OpenWebText2 32925
PhilPapers 68
Pile-CC 52790
PubMed Abstracts 29895
PubMed Central 5911
StackExchange 30378
USPTO Backgrounds 11415
Ubuntu IRC 22
Wikipedia (en) 17511
YoutubeSubtitles 342


In [7]:
sub_dataset_indices = np.where(data_split == 'Enron Emails')[0]
sub_dataset = ds.select(sub_dataset_indices)

In [8]:
from load import *
model = load_model('pythia-70m') # only need tokenizer so load smallest model

Using pad_token, but it is not set yet.


Loaded pretrained model pythia-70m into HookedTransformer


In [9]:
from transformer_lens.utils import tokenize_and_concatenate
tds = tokenize_and_concatenate(sub_dataset, model.tokenizer)

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

#4:   0%|          | 0/1 [00:00<?, ?ba/s]

#5:   0%|          | 0/1 [00:00<?, ?ba/s]

#6:   0%|          | 0/1 [00:00<?, ?ba/s]

#7:   0%|          | 0/1 [00:00<?, ?ba/s]

#8:   0%|          | 0/1 [00:00<?, ?ba/s]

#9:   0%|          | 0/1 [00:00<?, ?ba/s]

In [15]:
(tds['tokens'][:, 1:] == 0).numpy().mean()

0.0

In [18]:
fds = load_feature_dataset('pile_test.pyth.512.-1')

/Users/wesgurnee/Documents/mechint/sparse_probing/sparse-probing/sparprob/lib/python3.9/site-packages/datasets/arrow_dataset.py:1533: FutureWarning: 'fs' was is deprecated in favor of 'storage_options' in version 2.8.0 and will be removed in 3.0.0.
You can remove this warning by passing 'storage_options=fs.storage_options' instead.
  warnings.warn(


In [19]:
fds

Dataset({
    features: ['tokens', 'distribution'],
    num_rows: 649011
})

In [21]:
649011 * 511

331644621

In [23]:
fds[1]

{'tokens': tensor([    0,    15,    72,   904,   253,  2281,   273,  3116,   273,   841,
         29111,   275,   253,  5928,   273,   824, 14221,   513,   417,  5194,
           342,   326,  2326,   275,  3753,   575,   683,    49,   360,   343,
          6961,  1570,  8079,    13,   253,  8813,   273,   824,  5233,  3116,
           949, 40405,  6466,    13,  5183,  1008,    84,  1504, 29982, 12430,
            28,  1214,    35,   886,  6961,    28,  1214,  2374, 31384,  6620,
            62,   285, 18755,   275,  2426,   273,  2685,  9990,  1008,    87,
           706,    76,  6968,   282,  6759,    28,  1214,    49,   360,   343,
          6961,    28,  1214,    42, 10443,  6961,  1092,   310, 26415,   275,
           253,  3745,   273, 33217, 14221,   281,  7278,   253,  2281,   273,
         22959,   875, 15935,  8357,    14,  2003,  3741,    15,   187,   187,
            34,  4619,  8900,    13,  1955,   281, 40392,    13,   475,   292,
           355,  5929,   575,   683,    35